### 네이버 검색결과 크롤링 - 수원 맛집

In [37]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup


In [38]:
url = 'https://search.naver.com/search.naver?where=view&sm=tab_jum&query='+quote('수원맛집')
res = requests.get(url)
soup = BeautifulSoup(res.text)

In [39]:
lis = soup.select('.bx._svp_item')
len(lis)

30

- selenium으로 데이터 가져오기

In [40]:
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe', options=options)
driver.get(url)

In [41]:
# View 페이지를 아래로 내려 많은 데이터 확보
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

body = driver.find_element(By.TAG_NAME, 'body')
for _ in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [42]:
soup = BeautifulSoup(driver.page_source)
lis = soup.select('.bx._svp_item')
len(lis)

95

In [43]:
driver.close()

- 날짜, 제목, 컨텐츠

In [44]:
li = lis[0]
date = li.select_one('.sub_time.sub_txt').get_text().strip()
date

'2023.04.05.'

In [45]:
title = li.select_one('.api_txt_lines.total_tit._cross_trigger').get_text().strip()
title

'일품이던 수원맛집베스트10'

In [46]:
content = li.select_one('.api_txt_lines.dsc_txt').get_text().strip()
content[:200]

'하러 수원맛집베스트10에 속한다는 갈비집을 다녀왔어요. 동네 주민들 뿐만 아니라 멀리서도 많은 사람들이 찾아가는 곳이라는데 소문대로 고퀄리티의 식사를 즐길 수 있던 곳이었어요. 고반가든 서수원점 주소 : 경기...'

- 날짜 처리

In [47]:
from datetime import datetime, timedelta

today = datetime.now()
today

datetime.datetime(2023, 5, 31, 13, 51, 24, 309384)

In [48]:
# 어제
date = today - timedelta(1)
f'{date.year}.{date.month}.{date.day}'

'2023.5.30'

In [49]:
# 23시간 전
date = today - timedelta(hours=23)
date.strftime('%Y.%m.%d')

'2023.05.30'

In [50]:
# 2일 전
date = today - timedelta(2)
f'{date.year}.{date.month}.{date.day}'

'2023.5.29'

In [51]:
# 45분 전
date = today - timedelta(minutes=45)
f'{date.year}.{date.month}.{date.day}'

'2023.5.31'

In [52]:
def conver_date(s):
    if len(s) == 11:
        return s[:-1]    
    today = datetime.now()
    if '어제' in s:
        date = today - timedelta(1)
    elif '일' in s:
        date = today - timedelta(int(s[0]))
    elif '시간' in s:
        date = today - timedelta(hours=int(s[:-4]))
    elif '분' in s:
        date = today - timedelta(minutes=int(s[:-3]))

    return date.strftime('%Y.%m.%d')

In [53]:
lines = []
for li in lis:
    date = li.select_one('.sub_time.sub_txt').get_text().strip()
    date = conver_date(date)
    title = li.select_one('.api_txt_lines.total_tit._cross_trigger').get_text().strip()
    content = li.select_one('.api_txt_lines.dsc_txt').get_text().strip()
    lines.append({'날짜':date, '제목':title, '내용':content})

In [54]:
df = pd.DataFrame(lines)
df.tail()

,날짜,제목,내용
90,2023.05.23,[수원 맛집][행궁동 맛집] 수원 카이센동맛집은 마츠리동,수원맛집 / 수원일식 /수원행궁동맛집 /행궁동맛집/ 행궁동일식 /행궁동맛집추천 /행...
91,2023.05.17,수원맛집 월드컵공원 데이트 코스 낙원갈비집,가능 수원맛집 낙원갈비집 수원월드컵점은 수원월드컵경기장 바로 앞에 위치한 갈비맛집인...
92,2023.05.24,수원 맛집의 진득한 국물이 좋은 수원 온미관~,주소: 경기 수원시 영통구 메탄로 108번 길 52 전화번호: 0507- 1443-...
93,2023.04.27,수원 맛집 계동돼지 인계동 맛집,계동돼지 수원 맛집&인계동 맛집 일-목 16:00-24:00 금-토 16:00-02...
94,2023.05.25,"수원닭갈비가 생각날때마다 찾는 수원맛집, 춘계옥 본점",되었어요~ #수원맛집 #수원닭갈비 #수원고기집 #수원고깃집 #수원회식 #수원모임 #...
